# Chains
#### What is a Chain?

A chain in LangChain is a pipeline of components where the output of one step automatically becomes the input of the next. Instead of manually connecting functions together, LangChain standardizes the way prompts, models, parsers, and retrievers interact. 

#### Chains vs Agents

Chains and agents both help coordinate multiple LLM calls, but they differ in behavior. 
* A chain follows a fixed path that you define at design time. 
* An agent is dynamic: it decides at runtime which tool or path to take, depending on the input.

Chains are simpler, more deterministic, while agents provide flexibility at the cost of predictability

#### Why Chains Matter

* Reusability,
* Maintainability
* Possible to add retries, caching, fallbacks or tracing with out changing the main workflow logic
* Enforce consistency



## Langchain Expression Language
**The LangChain Expression Language (LCEL)** is a declarative way to compose components. It was developed to address limitations of the old Chain class system. LCEL provides several advantages:
* **Streaming Support**: LCEL makes it easier to stream outputs from each step of your sequence, even when you are chaining calls. The older chains required using callbacks for streaming.
* **Parallelism**: You can easily run multiple parts of a chain in parallel with LCEL, which can improve the performance of your application.
* **Debugging and Visibility**: LCEL provides better  visibility into the structure of your application, making it easier to debug and inspect the intermediate steps of a chain.
* **Standard Interface**: LCEL introduced the Runnable protocol, a standard interface that all components adhere to. SimpleSequentialChain was an early implementation of this concep

LCEL is the modern standard for building chains. Its key idea is `composability`: every component in LangChain implements a `Runnable` interface, and you can combine them using the pipe operator `(|)`. This makes chain definitions short and expressive.

### RunnableSequence (Sequential Chains)
A `RunnableSequence` is the simplest and most common type of chain in LangChain. It defines a step-by-step pipeline where the output of one component becomes the input of the next. Think of it as a conveyor belt: input enters, gets transformed by each stage in order, and finally produces the output.

In LCEL, the most natural way to build a sequential chain is with the `|` operator, which implicitly creates a RunnableSequence

#### Simple Example

In [39]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser
from dotenv import load_dotenv

In [29]:
load_dotenv()
llm = ChatOpenAI(model="gpt-4o", temperature=0.7, api_key=os.getenv("OPEN_API_KEY"))

In [30]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly assistant."),
    ("human", "Tell me a joke about {topic}")
])

parser = StrOutputParser()

chain = prompt | llm | parser
# chain = RunnableSequence([prompt, llm, parser])

result = chain.invoke({"topic": "LangChain"})
print(result)


Why don't LangChain models ever get lost?

Because they always follow the "chain of command"!


#### Using JsonOutputParser

In [31]:
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")

parser = JsonOutputParser(pydantic_object=Joke)
format_instructions = parser.get_format_instructions()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a comedian AI. Output only valid JSON."),
    ("human", "Write a joke about {topic}")
])

chain = prompt | llm | parser

print(chain.invoke({"topic": "LangChains"}))

{'joke': 'Why did the LangChain developer bring a ladder to the library? Because they heard the books were stacked with high-level abstractions!'}


In [32]:
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")

parser = JsonOutputParser(pydantic_object=Joke)
format_instructions = parser.get_format_instructions()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a comedian AI. Output only valid JSON."),
    ("human", "Write a joke about {topic}\n"
    "{format_instructions}")
])

chain = prompt | llm | parser

print(chain.invoke({"topic": "LangChains", "format_instructions": format_instructions}))

{'setup': 'Why did the LangChain break up with its library?', 'punchline': "It couldn't handle the lack of support!"}


#### Expanding with More Steps

Since steps are modular, you can insert transformations before or after the LLM. For example, preprocessing input to uppercase before sending to the model, or postprocessing to count tokens.

In [45]:
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")

parser = PydanticOutputParser(pydantic_object=Joke)
format_instructions = parser.get_format_instructions()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a comedian AI. Output only valid JSON."),
    ("human", "Write a joke about {topic}\n"
    "{format_instructions}")
])

chain = prompt | llm | parser

print(chain.invoke({"topic": "LangChains", "format_instructions": format_instructions}))

def to_upper(inputs: dict) -> dict:
    inputs["topic"].upper()
    return inputs

# uppercase = RunnableLambda(lambda x: {**x, "topic": x["topic"].upper()})

def add_exclamation(output: Joke) -> str:
    output.setup = output.setup + " 🚀"
    output.punchline = output.setup + " "
    return output

chain = RunnableSequence(to_upper, prompt, llm, parser, add_exclamation)

print(chain.invoke({"topic": "LangChain", "format_instructions": format_instructions}))


setup='Why did the LangChain break up with its AI partner?' punchline="It couldn't handle the lack of context in their relationship!"
setup="Why don't LangChain developers play hide and seek? 🚀" punchline="Why don't LangChain developers play hide and seek? 🚀 "


### RunnableMap
A `RunnableMap` is a chain that executes multiple branches in parallel. Instead of sending your input through one fixed pipeline, you can fork the input into several independent sub-chains at the same time. Each sub-chain processes the same input (or a portion of it), and the results are collected into a dictionary keyed by branch names.

This pattern is useful when you want to extract multiple types of information from the same input in a single cal
##### Example
Generating a summary, extracting keywords, and classifying sentiment all at once.

In [46]:
from langchain_core.runnables import RunnableMap

In [47]:
# Branch 1: Summarization
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a summarizer."),
    ("human", "Summarize the following text:\n\n{text}")
])
summary_chain = summary_prompt | llm | StrOutputParser()

# Branch 2: Keyword extraction
keyword_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a keyword extractor."),
    ("human", "Extract 5 keywords from the following text:\n\n{text}")
])
keyword_chain = keyword_prompt | llm | StrOutputParser()

# RunnableMap to run both in parallel
map_chain = RunnableMap({
    "summary": summary_chain,
    "keywords": keyword_chain
})

doc = "LangChain is a framework that helps developers build applications with large language models by providing composable abstractions."
result = map_chain.invoke({"text": doc})
print(result)


{'summary': 'LangChain is a framework designed to assist developers in creating applications using large language models by offering composable abstractions.', 'keywords': '1. LangChain\n2. Framework\n3. Developers\n4. Applications\n5. Language Models'}


#### Note 
Branches don’t need to return the same type. One branch can return a string, another a list, another a JSON object. The results will still be collected into a dictionary.

**For example**: you could have:

* summary → plain string
* keywords → list of strings
* classification → structured JSON (via Pydantic)

This flexibility makes RunnableMap powerful for building pipelines that produce rich, multi-field outputs in a single run.

In [48]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

# Branch 1: Summarization
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a summarizer."),
    ("human", "Summarize the following text:\n\n{text}")
])
summary_chain = summary_prompt | llm | StrOutputParser()

# Branch 2: Keyword extraction
keyword_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a keyword extractor."),
    ("human", "Extract 5 keywords from the following text:\n\n{text}")
])
keyword_chain = keyword_prompt | llm | StrOutputParser()

# Helper: count words in a string
def word_count_text(inputs: dict) -> int:
    return len(inputs["text"].split())

def word_count_summary(inputs: dict) -> int:
    return len(inputs["summary"].split())

# RunnableMap to run all in parallel
map_chain = RunnableMap({
    "summary": summary_chain,
    "keywords": keyword_chain,
    "original_word_count": word_count_text,
    # NOTE: this depends on summary, so we’ll wrap with a lambda inside another map
})

# First, run summary + keywords + original_word_count
partial_result = map_chain.invoke({"text": doc})

# Then compute word_count of the summary
partial_result["summary_word_count"] = len(partial_result["summary"].split())

print(partial_result)


{'summary': 'LangChain is a framework designed to assist developers in creating applications using large language models by offering modular and composable abstractions.', 'keywords': '1. LangChain  \n2. framework  \n3. developers  \n4. applications  \n5. large language models', 'original_word_count': 17, 'summary_word_count': 21}


### Hierarchical composition
This is the idea that you can nest maps and sequences inside each other to build multi-level pipelines.

In [49]:
# --- Branch 1: summary ---
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a summarizer."),
    ("human", "Summarize the following text:\n\n{text}")
])
summary_chain = summary_prompt | llm | StrOutputParser()

# --- Branch 2: keywords ---
keyword_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a keyword extractor."),
    ("human", "Extract 5 keywords from the following text:\n\n{text}")
])
keyword_chain = keyword_prompt | llm | StrOutputParser()

# --- Parallel fan-out (both branches see the same input dict) ---
map_chain = RunnableMap({
    "summary": summary_chain,
    "keywords": keyword_chain,
    "original_word_count": lambda inputs: len(inputs["text"].split()),
})

# --- Post-process step that depends on the summary produced by map_chain ---
def add_summary_wc(outputs: dict) -> dict:
    # outputs looks like: {"summary": "...", "keywords": "...", "original_word_count": int}
    return {
        **outputs,
        "summary_word_count": len(outputs["summary"].split())
    }

# Option A: Explicit RunnableSequence
seq = RunnableSequence(map_chain, add_summary_wc)

# Option B: LCEL pipe shorthand (equivalent)
# seq = map_chain | add_summary_wc

doc = "LangChain is a framework that helps developers build applications with large language models by providing composable abstractions."
result = seq.invoke({"text": doc})
print(result)


{'summary': 'LangChain is a framework designed to assist developers in creating applications using large language models by offering composable abstractions.', 'keywords': '1. LangChain\n2. framework\n3. developers\n4. applications\n5. language models', 'original_word_count': 17, 'summary_word_count': 19}


# Depricated

## LLMChain (depricated)
* LLMChain is a foundational building block or component within the LangChain framework that combines a Large Language Model (LLM) with a prompt template.
  * **Format user input**: The prompt template structures the input for the LLM.
  * **Call the LLM**: The formatted prompt is sent to the LLM for processing.
  * **Parse the output**: The raw response from the LLM is transformed into a more usable format by the output parser.
* Deprecated since version 0.1.17

In [37]:
from pydantic import BaseModel, Field
from typing import List
import json
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [38]:
# Pydantic model for structured output
class MovieSuggestion(BaseModel):
    title: str = Field(description="The title of the movie.")
    genre: str = Field(description="The genre of the movie.")
    reason: str = Field(description="A brief explanation of why the movie was suggested.")

# Set up the parser
parser = PydanticOutputParser(pydantic_object=MovieSuggestion)

# Define the message templates
system_message_template = SystemMessagePromptTemplate.from_template(
    "You are a helpful AI assistant that recommends a movie based on the user's mood. "
    "Always provide a single recommendation."
)
human_message_template = HumanMessagePromptTemplate.from_template(
    "I'm in the mood for something {mood}. I want my recommendation to be a {genre}. "
    "\n\n{format_instructions}"
)

# Combine the message templates into a ChatPromptTemplate
chat_template = ChatPromptTemplate.from_messages([
    system_message_template,
    human_message_template
])

# Create the LLMChain with the ChatPromptTemplate
# The output parser is passed to the chain constructor.
# This approach is less elegant than LCEL.
chain = LLMChain(
    llm=llm,
    prompt=chat_template,
    output_parser=parser
)

# Define the input variables for the human message and format instructions
input_data = {
    "mood": "adventurous",
    "genre": "action",
    "format_instructions": parser.get_format_instructions()
}
# Invoke the chain
# The `.invoke()` method of LLMChain returns a dictionary, so we access the output via the "text" key.
# The output is a Pydantic object, thanks to the parser.
result = chain.invoke(input_data)

# The output from the chain is automatically parsed into our Pydantic model.
movie_suggestion = result['text']

# Print the result
print("Chain output type:", type(movie_suggestion))
print(movie_suggestion)
print(f"\nTitle: {movie_suggestion.title}")
print(f"Genre: {movie_suggestion.genre}")
print(f"Reason: {movie_suggestion.reason}")

Chain output type: <class '__main__.MovieSuggestion'>
title='Mad Max: Fury Road' genre='Action' reason='This movie is a high-octane action adventure set in a post-apocalyptic world, filled with thrilling chase sequences and explosive battles, making it perfect for an adventurous mood.'

Title: Mad Max: Fury Road
Genre: Action
Reason: This movie is a high-octane action adventure set in a post-apocalyptic world, filled with thrilling chase sequences and explosive battles, making it perfect for an adventurous mood.


## Simple Sequential Chain and Sequential Chain  (depricated)
A sequential chain is more advanced and links several steps together, where the output of one step becomes the input for the next

#### Example: Blog content generation workflow
* **Chain 1**: Takes a topic and generates a bullet-point outline.
* **Chain 2**: Takes the outline and writes a full blog post.
* **Chain 3**: Takes the blog post and creates a
summary

**Note**: SequentialChain and SimpleSequentialChain is deprecated in favor of the LangChain Expression Language (LCEL).

<table>
      <thead>
        <tr>
          <th>Feature</th>
          <th>SimpleSequentialChain</th>
          <th>SequentialChain</th>
        </tr>
      </thead>
      <tbody>
        <tr>
          <td>Input/Output</td>
          <td>Each step has a single input and a single output. <br>The output of one step is implicitly passed as the input to the next.</td>
          <td>Allows multiple named inputs and outputs at each step, <br>giving you more explicit control over data flow.</td>
        </tr>
         <tr>
          <td>Workflow</td>
          <td>Ideal for simple, linear workflows where each<br> step is directly dependent on the output of the previous one. <br>This is sufficient for many basic use cases.</td>
          <td> step is directly dependent on the output<br> of the previous one. This is sufficient for many basic use cases.<br>	Built for complex workflows where you might need to use<br> the output from a previous step in multiple subsequent steps,<br> or combine multiple inputs before a step.</td>
        </tr>
         <tr>
          <td>Variable Passing</td>
          <td>Automatically handles the variable <br>passing between steps, making it simple<br> to set up. It is less flexible but requires less configuration.</td>
          <td>Requires you to explicitly define and <br>map the input and output variables for each step.<br> This offers greater flexibility and control over how data moves through the chain.</td>
        </tr>
         <tr>
          <td>Use case</td>
          <td>A simple step process</td>
          <td>A multi-step process with branching logic</td>
        </tr>
      </tbody>
</table>

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import SimpleSequentialChain

In [ ]:
# 2. Define the first chain: Create an outline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt_outline = PromptTemplate.from_template(
    "Create a bullet-point outline for a blog post about {topic}."
)
chain_outline = LLMChain(llm=llm, prompt=prompt_outline)


# 3. Define the second chain: Write the blog post
prompt_post = PromptTemplate.from_template(
    "Write a detailed blog post based on this outline: {outline}"
)
chain_post = LLMChain(llm=llm, prompt=prompt_post)


# 4. Define the third chain: Summarize the post
prompt_summary = PromptTemplate.from_template(
    "Summarize this blog post: {post}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary)


# 5. Combine the chains into a SimpleSequentialChain
final_chain = SimpleSequentialChain(
    chains=[chain_outline, chain_post, chain_summary], verbose=True
)

# 6. Invoke the final chain
final_result = final_chain.invoke("using AI in art")
print("\n--- Final Summary ---")
print(final_result)

## Langchain Expression Language
**The LangChain Expression Language (LCEL)** is a declarative way to compose components. It was developed to address limitations of the old Chain class system. LCEL provides several advantages:
* **Streaming Support**: LCEL makes it easier to stream outputs from each step of your sequence, even when you are chaining calls. The older chains required using callbacks for streaming.
* **Parallelism**: You can easily run multiple parts of a chain in parallel with LCEL, which can improve the performance of your application.
* **Debugging and Visibility**: LCEL provides better  visibility into the structure of your application, making it easier to debug and inspect the intermediate steps of a chain.
* **Standard Interface**: LCEL introduced the Runnable protocol, a standard interface that all components adhere to. SimpleSequentialChain was an early implementation of this concep

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# 1. Define the LLM and output parser
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
parser = StrOutputParser()

# 2. Define the prompt templates
prompt_outline = ChatPromptTemplate.from_template(
    "Create a bullet-point outline for a blog post about {topic}."
)

prompt_post = ChatPromptTemplate.from_template(
    "Write a detailed blog post based on this outline: {outline}"
)

prompt_summary = ChatPromptTemplate.from_template(
    "Summarize this blog post: {post}"
)
# 3. Define the individual chains using LCEL
chain_outline = prompt_outline | llm | parser
chain_post = prompt_post | llm | parser
chain_summary = prompt_summary | llm | parser

# 4. Construct the overall chain
overall_chain = (
    # Step 1: Generate the outline
    # Pass through the original input `topic` and add the `outline` key
    RunnablePassthrough.assign(outline=chain_outline)
    # Step 2: Generate the post
    # Pass through the new input (with 'topic' and 'outline') and add the 'post' key
    | RunnablePassthrough.assign(post=chain_post)
    # Step 3: Generate the summary
    # Pass through the input (with 'topic', 'outline', and 'post') and add the 'summary' key
    | RunnablePassthrough.assign(summary=chain_summary)
)
# 5. Invoke the chain
result = overall_chain.invoke({"topic": "LangChain Expression Language (LCEL)"})

# 6. Access the results
print("--- Outline ---")
print(result["outline"])
print("\n--- Blog Post ---")
print(result["post"])
print("\n--- Summary ---")
print(result["summary"])

In [ ]:
# Set up the parser
parser = PydanticOutputParser(pydantic_object=MovieSuggestion)

# Define the chat prompt template with multiple message roles
chat_template = ChatPromptTemplate.from_messages(
    [
        # The system message sets the overall behavior and instructions
        ("system", "You are a helpful AI assistant that recommends a movie based on the user's mood. Always provide a single recommendation. {format_instructions}"),

        # This is a placeholder for previous conversation turns (optional)
        # It's not strictly a part of the *current* prompt but shows how history is inserted
        # from langchain.prompts import MessagesPlaceholder
        # MessagesPlaceholder(variable_name="chat_history")

        # Example of a previous conversation turn (simulating what might have happened)
        # This is a static example, but in a real app, you'd insert actual history here
        ("ai", "I'm good, thanks for asking! What kind of mood are you in?"),

        # Template for the current user's input
        ("human", "I'm in the mood for something {mood}. I want my recommendation to be a {genre}."),
    ]
)

# Initialize the chat model (e.g., GPT-4)
# You need to have your OpenAI API key set as an environment variable
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

# Create the chain using LCEL syntax
# This is the modern, more robust alternative to LLMChain
chain = chat_template | llm | parser

# Define the input variables
input_data = {"mood": "adventurous", "genre": "action", "format_instructions": parser.get_format_instructions()}

# Invoke the chain to get the structured output
result = chain.invoke(input_data)

# Print the result
print(result)
print(f"\nTitle: {result.title}")
print(f"Genre: {result.genre}")

## Route Chain

In [ ]:
from typing import Literal, Dict, Callable
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [ ]:
def make_simple_chain(system_msg: str):
    prompt = ChatPromptTemplate.from_messages([("system", system_msg), ("user", "{input}")])
    return prompt | llm | StrOutputParser()

destinations: Dict[str, Callable] = {
    "math":    make_simple_chain("You are a careful math solver."),
    "code":    make_simple_chain("You are a helpful coding assistant. Provide runnable code."),
    "general": make_simple_chain("You are a concise general assistant.")
}

# ----- reusable router (no format_instructions needed) -----
class Route(BaseModel):
    destination: Literal[tuple(destinations.keys())] = Field(
        description="Best handler for the user's query."
    )
    reason: str

router_system = (
    "You are a router. Pick exactly one destination for the user's query "
    f"from: {', '.join(destinations.keys())}."
)

router_prompt = ChatPromptTemplate.from_messages(
    [("system", router_system), ("user", "{input}")]
)

# 'with_structured_output' makes the LLM return a Route object via native tool/JSON
router_chain = router_prompt | llm.with_structured_output(Route)

# ----- branch wiring (tiny and reusable) -----
def make_routed_chain():
    router_and_input = RunnableMap(route=router_chain, input=lambda x: x["input"])
    return router_and_input | RunnableBranch(
        *((lambda x, k=k: x["route"].destination == k, v) for k, v in destinations.items()),
        # default fallback:
        destinations["general"]
    )

routed_chain = make_routed_chain()

# use:
routed_chain.invoke({"input": "Derivative of x^3 + 2x?"})
routed_chain.invoke({"input": "Write a Python function to merge two sorted lists."})
